In [ ]:
# 📦 Install required packages (if not already installed)
# !pip install openai pandas geopy

from openai import OpenAI
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import json

# 🔐 Initialize OpenAI client
# client = OpenAI(api_key="")  # Replace with your OpenAI API key

# 📄 Load the food bank dataset
data = pd.read_csv("data.csv")  # Ensure this is the same file used in your AWS deployment

# 🧠 Step 1: Get user input
user_query = "I need food banks near 4700 Berwyn House Rd, college park, MD that are open on Thursdays and have walk-up distribution."  # Replace with actual user input that are open on Thursdays and have walk-up distribution.

# 🧾 Step 2: Feature Extraction Prompt (LLM1)
feature_extractor_prompt = f"""
Extract the following information from the user's query:
- Address
- Region (DC, MD, VA)
- County
- Day of the week
- Distribution mode (e.g., Walk Up, Drive Through)
- A cleaned version of the address for geocoding

Query: "{user_query}"

Return the result as a valid JSON object with these keys only:
address, region, county, day, mode, geo_address
"""

response1 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": feature_extractor_prompt}]
)

# ✅ Clean and parse JSON output
raw_content = response1.choices[0].message.content.strip()
if raw_content.startswith("```json"):
    raw_content = raw_content.lstrip("```json").rstrip("```").strip()
elif raw_content.startswith("```"):
    raw_content = raw_content.lstrip("```").rstrip("```").strip()

features = json.loads(raw_content)

print("🔍 Extracted Features:")
print(features)


🔍 Extracted Features:
{'address': '4700 Berwyn House Rd, College Park, MD', 'region': 'MD', 'county': "Prince George's", 'day': 'Thursday', 'mode': 'Walk Up', 'geo_address': '4700 Berwyn House Rd, College Park, MD'}


In [18]:
# Filter by region and county
filtered_data = data[
    (data['Region'].str.upper() == features['region'].upper()) &
    (data['Zone'].str.upper() == features['county'].upper())
]

# Geocode address using geopy + Nominatim
geolocator = Nominatim(user_agent="foodbank_chatbot")
user_location = geolocator.geocode(features["geo_address"])
user_coords = (user_location.latitude, user_location.longitude)

# Calculate distances
def compute_distance(row):
    bank_coords = (row['Latitude'], row['Longitude'])
    return geodesic(user_coords, bank_coords).miles

# Compute distances and store in a new list
distances = []
for _, row in filtered_data.iterrows():
    try:
        bank_coords = (row['Latitude'], row['Longitude'])
        dist = geodesic(user_coords, bank_coords).miles
        distances.append(dist)
    except:
        distances.append(None)  # or float('inf') if you want to deprioritize it

# Assign the result as a new column
filtered_data = filtered_data.copy()
filtered_data["Distance"] = distances
top_nearby = filtered_data.sort_values('Distance').head(10)

print("📍 Top Nearby Food Banks:")

if filtered_data.empty:
    print("❗ No matching food banks found for that region/county. Try relaxing the filters.")
else:
    print(top_nearby[['Name', 'Address', 'Phone', 'Distance']])


📍 Top Nearby Food Banks:
❗ No matching food banks found for that region/county. Try relaxing the filters.
